In [1]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import scipy
import librosa

from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd
from tensorflow.contrib import signal

import python_speech_features

# Data Loading
import re
from glob import glob

# Prediction
from tqdm import tqdm

# Layers
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib.framework import arg_scope
from tflearn.layers.conv import global_avg_pool
from tensorflow.contrib.layers import batch_norm, flatten

# it's a magic function :)
# Training
from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn

import cv2


# #KERAS

# import os
# os.environ['KERAS_BACKEND'] = 'tensorflow'
# # os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

# import keras
# import numpy as np
# from keras.models import Model, load_model
# from keras.layers import Dense, Input, Activation, Flatten, Add
# from keras.layers import BatchNormalization
# from keras.regularizers import l2
# from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# from keras.datasets import cifar10
# from keras.utils import np_utils
# from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
# from keras.preprocessing.image import ImageDataGenerator

# from keras import backend
# import matplotlib.pyplot as plt


%matplotlib inline

In [2]:
DATADIR = './data' # unzipped train and test data
OUTDIR = './model-k' # just a random name

POSSIBLE_LABELS = 'yes no up down left right on off stop go silence unknown'.split()
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}
name2id = {name: i for i, name in id2name.items()}


def load_data(data_dir):
    """ Return 2 lists of tuples:
    [(class_id, user_id, path), ...] for train
    [(class_id, user_id, path), ...] for validation
    """
    # Just a simple regexp for paths with three groups:
    # prefix, label, user_id
    pattern = re.compile("(.+\/)?(\w+)\/([^_]+)_.+wav")
    all_files = glob(os.path.join(data_dir, 'train/audio/*/*wav'))

    with open(os.path.join(data_dir, 'train/validation_list.txt'), 'r') as fin:
        validation_files = fin.readlines()
    valset = set()
    for entry in validation_files:
        r = re.match(pattern, entry)
        if r:
            valset.add(r.group(3))

    possible = set(POSSIBLE_LABELS)
    train, val, bg_noise = [], [], []
    for entry in all_files:
        r = re.match(pattern, entry)
        if r:
            label, uid = r.group(2), r.group(3)
            if label == '_background_noise_':
                bg_noise.append(entry)
                label = 'silence'
            if label not in possible:
                label = 'unknown'

            label_id = name2id[label]

            sample = (label_id, uid, entry)
            if uid in valset:
                val.append(sample)
            else:
                train.append(sample)

    print('There are {} train, {} val, and {} bg noise samples'.format(len(train), len(val), len(bg_noise)))
    return train, val, bg_noise

trainset, valset, noiseset = load_data(DATADIR)

There are 57929 train, 6798 val, and 6 bg noise samples


In [35]:
def wav_read(fname):
    wav, _ = librosa.load(fname, sr=None)
    return wav

def normalize_audio(wav):
    return wav/max(wav)

def time_shift(wav, shift):
    start_ = int(shift)
    if start_ >= 0:
        wav_time_shift = np.r_[wav[start_:], np.random.uniform(-0.001,0.001, start_)]
    else:
        wav_time_shift = np.r_[np.random.uniform(-0.001,0.001, -start_), wav[:start_]]
    return normalize_audio(wav_time_shift)

def speed_change(wav, speed_rate):
    # rate: lower is faster
    wav_speed_tune = cv2.resize(wav, (1, int(len(wav) * speed_rate))).squeeze()
    if len(wav_speed_tune) < 16000:
        pad_len = 16000 - len(wav_speed_tune)
        wav_speed_tune = np.r_[np.random.uniform(-0.001,0.001,int(pad_len/2.)),
                               wav_speed_tune,
                               np.random.uniform(-0.001,0.001,int(np.ceil(pad_len/2.)))]
    else: 
        cut_len = len(wav_speed_tune) - 16000
        wav_speed_tune = wav_speed_tune[int(cut_len/2.):int(cut_len/2.)+16000]
    return normalize_audio(wav_speed_tune)

def noise_add(wav, percent, ind):
#     bg, sr = librosa.load(noiseset[ind], sr=None)
    bg = wav_read(noiseset[ind])
    bg = normalize_audio(bg)
    start_ = np.random.randint(bg.shape[0]-16000)
    bg_slice = bg[start_ : start_+16000]
    wav_with_bg = wav * percent + bg_slice * (1-percent)
    return normalize_audio(wav_with_bg)

def get_spectrogram(wav):
    v = 600
    D = librosa.stft(wav, n_fft=v, hop_length=50,
                     win_length=v, window='hamming')
    spect, phase = librosa.magphase(D)
    spect = scipy.ndimage.zoom(spect,1./7, order=1)
    spect = spect.reshape(np.expand_dims(spect, axis=2).shape)
    return spect

def frange(x, y, jump):
    while x < y:
        yield x
        x += jump
        
def all_aug(wav, params):
    time, speed, noise_percent, noise_ind = params
    aug = wav
    aug = time_shift(aug, time)
    aug = speed_change(wav, speed)
    if noise_ind == -1:
        aug = noise_add(aug, 1, noise_ind)
    else:
        aug = noise_add(aug, noise_percent, noise_ind)
#     print aug.dtype
    return aug.astype(np.float32)

In [36]:
DIV = 100
def data_generator(data, params, mode='train'):
    def generator():
        count = 0
        if mode == 'train':
            np.random.shuffle(data)
        
        for (label_id, uid, fname) in data:
            try:
                wav = wav_read(fname)
                wav = normalize_audio(wav)
        
                L = 16000  # be aware, some files are shorter than 1 sec!
                if len(wav) < L:
                    pad_len = L - len(wav)
                    wav = np.r_[np.random.uniform(-0.001,0.001,int(pad_len/2.)),
                                wav,
                                np.random.uniform(-0.001,0.001,int(np.ceil(pad_len/2.)))]
                elif len(wav) > L:
                    cut_len = len(wav) - L
                    wav = wav[int(cut_len/2.):int(cut_len/2.)+L]
                    
                if mode == 'train':
                    
                    variations = []

                    time_shift_range = 4000
                    for time in xrange(-time_shift_range, time_shift_range+1, 1000):
                        for speed in frange(0.4,1.7,0.3):
                            for noise_percentage in frange(0.2,1,0.1):
                                for noise_ind in xrange(-1,len(noiseset)):
                                    variation = [time, speed, noise_percentage, noise_ind]
                                    variations.append(variation)

                    len_var = len(variations)
                    selected_indices = np.random.choice(len_var, int(len_var/DIV))

                    for ind in selected_indices:
                        yield dict(
                            target=np.int32(label_id),
                            wav=all_aug(wav, variations[ind])[:16000]
                        )
                else:
                    yield dict(
                        target=np.int32(label_id),
                        wav=wav[:16000],
                    )
                                
            except Exception as err:
                print(err, label_id, uid, fname)

    return generator

In [37]:
import tensorflow as tf
from tensorflow.contrib import layers

def baseline(x, params, is_training):
    x = layers.batch_norm(x, is_training=is_training)
    for i in range(4):
        x = layers.conv2d(
            x, 16 * (2 ** i), 3, 1,
            activation_fn=tf.nn.elu,
            normalizer_fn=layers.batch_norm if params.use_batch_norm else None,
            normalizer_params={'is_training': is_training}
        )
        x = layers.max_pool2d(x, 2, 2)

    # just take two kind of pooling and then mix them, why not :)
    mpool = tf.reduce_max(x, axis=[1, 2], keep_dims=True)
    apool = tf.reduce_mean(x, axis=[1, 2], keep_dims=True)

    x = 0.5 * (mpool + apool)
    # we can use conv2d 1x1 instead of dense
    x = layers.conv2d(x, 128, 1, 1, activation_fn=tf.nn.elu)
    x = tf.nn.dropout(x, keep_prob=params.keep_prob if is_training else 1.0)
    
    # again conv2d 1x1 instead of dense layer
    logits = layers.conv2d(x, params.num_classes, 1, 1, activation_fn=None)
    return tf.squeeze(logits, [1, 2])

In [38]:
from tensorflow.contrib import signal

# features is a dict with keys: tensors from our datagenerator
# labels also were in features, but excluded in generator_input_fn by target_key

def model_handler(features, labels, mode, params, config):
    # Im really like to use make_template instead of variable_scopes and re-usage
    extractor = tf.make_template(
        'extractor', baseline,
        create_scope_now_=True,
    )
    # wav is a waveform signal with shape (16000, )
    wav = features['wav']
    # we want to compute spectograms by means of short time fourier transform:
    specgram = signal.stft(
        wav,
        400,  # 16000 [samples per second] * 0.025 [s] -- default stft window frame
        160,  # 16000 * 0.010 -- default stride
    )
    # specgram is a complex tensor, so split it into abs and phase parts:
    phase = tf.angle(specgram) / np.pi
    # log(1 + abs) is a default transformation for energy units
    amp = tf.log1p(tf.abs(specgram))
    
    x = tf.stack([amp, phase], axis=3) # shape is [bs, time, freq_bins, 2]
    x = tf.to_float(x)  # we want to have float32, not float64

    logits = extractor(x, params, mode == tf.estimator.ModeKeys.TRAIN)

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        # some lr tuner, you could use move interesting functions
        def learning_rate_decay_fn(learning_rate, global_step):
            return tf.train.exponential_decay(
                learning_rate, global_step, decay_steps=10000, decay_rate=0.99)

        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=params.learning_rate,
            optimizer=lambda lr: tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True),
            learning_rate_decay_fn=learning_rate_decay_fn,
            clip_gradients=params.clip_gradients,
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

        specs = dict(
            mode=mode,
            loss=loss,
            train_op=train_op,
        )

    if mode == tf.estimator.ModeKeys.EVAL:
        prediction = tf.argmax(logits, axis=-1)
        acc, acc_op = tf.metrics.mean_per_class_accuracy(
            labels, prediction, params.num_classes)
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        specs = dict(
            mode=mode,
            loss=loss,
            eval_metric_ops=dict(
                acc=(acc, acc_op),
            )
        )

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'label': tf.argmax(logits, axis=-1),  # for probability just take tf.nn.softmax()
            'sample': features['sample'], # it's a hack for simplicity
        }
        specs = dict(
            mode=mode,
            predictions=predictions,
        )
    return tf.estimator.EstimatorSpec(**specs)


def create_model(config=None, hparams=None):
    return tf.estimator.Estimator(
        model_fn=model_handler,
        config=config,
        params=hparams,
    )

params=dict(
    seed=2018,
    batch_size=64,
    keep_prob=0.5,
    learning_rate=1e-3,
    clip_gradients=15.0,
    use_batch_norm=True,
    num_classes=len(POSSIBLE_LABELS),
)

hparams = tf.contrib.training.HParams(**params)
dir_path = os.path.join(OUTDIR, 'eval')

if not os.path.exists(dir_path): os.makedirs(dir_path)
model_dir = OUTDIR

run_config = tf.contrib.learn.RunConfig(model_dir=model_dir)

# it's a magic function :)
from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn
            
train_input_fn = generator_input_fn(
    x=data_generator(trainset, hparams, 'train'),
    target_key='target',  # you could leave target_key in features, so labels in model_handler will be empty
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)

val_input_fn = generator_input_fn(
    x=data_generator(valset, hparams, 'val'),
    target_key='target',
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)
            

def _create_my_experiment(run_config, hparams):
    exp = tf.contrib.learn.Experiment(
        estimator=create_model(config=run_config, hparams=hparams),
        train_input_fn=train_input_fn,
        eval_input_fn=val_input_fn,
        train_steps=10000, # just randomly selected params
        eval_steps=200,  # read source code for steps-epochs ariphmetics
        train_steps_per_iteration=1000,
    )
    return exp

tf.contrib.learn.learn_runner.run(
    experiment_fn=_create_my_experiment,
    run_config=run_config,
    schedule="continuous_train_and_eval",
    hparams=hparams)

INFO:tensorflow:Using config: {'_model_dir': './model-k', '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_session_config': None, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc36b9e20d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_log_step_count_steps': 100}
INFO:tensorflow:Training model for 1000 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./model-k/model.ckpt-10004
INFO:tensorflow:Saving checkpoints for 10005 into ./model-k/model.ckpt.
INFO:tensorflow:loss = 1.55184, step = 10005
INFO:tensorflow:Saving checkpoints for 10101 into ./model-k/model.ckpt.
INFO:tensorflow:gl

({'acc': 0.65106976, 'global_step': 11004, 'loss': 0.71565467}, [])

In [39]:
from tqdm import tqdm
# now we want to predict!
paths = glob(os.path.join(DATADIR, 'test/audio/*wav'))

def test_data_generator(data):
    def generator():
        for path in data:
            _, wav = wavfile.read(path)
            wav = wav.astype(np.float32) / np.iinfo(np.int16).max
            fname = os.path.basename(path)
            yield dict(
                sample=np.string_(fname),
                wav=wav,
            )

    return generator

test_input_fn = generator_input_fn(
    x=test_data_generator(paths),
    batch_size=hparams.batch_size, 
    shuffle=False, 
    num_epochs=1,
    queue_capacity= 10 * hparams.batch_size, 
    num_threads=1,
)

model = create_model(config=run_config, hparams=hparams)
it = model.predict(input_fn=test_input_fn)


# last batch will contain padding, so remove duplicates
submission = dict()
for t in tqdm(it):
    fname, label = t['sample'].decode(), id2name[t['label']]
    submission[fname] = label

with open(os.path.join(model_dir, 'submission.csv'), 'w') as fout:
    fout.write('fname,label\n')
    for fname, label in submission.items():
        fout.write('{},{}\n'.format(fname, label))

INFO:tensorflow:Using config: {'_model_dir': './model-k', '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_session_config': None, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc36b9e20d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_log_step_count_steps': 100}



0it [00:00, ?it/s]

INFO:tensorflow:Restoring parameters from ./model-k/model.ckpt-11004



Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/tommy/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/tommy/anaconda2/lib/python2.7/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/tommy/anaconda2/lib/python2.7/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration


1it [00:01,  1.59s/it]
65it [00:02, 24.65it/s]
129it [00:03, 35.57it/s]
193it [00:04, 41.93it/s]
257it [00:05, 46.17it/s]
321it [00:06, 49.15it/s]
385it [00:07, 51.29it/s]
449it [00:08, 52.94it/s]
513it [00:09, 54.13it/s]
577it [00:10, 55.08it/s]
641it [00:11, 56.08it/s]
705it [00:12, 56.59it/s]
769it [00:13, 57.25it/s]
833it [00:14, 57.88it/s]
897it [00:15, 58.46it/s]
961it [00:16, 58.96it/s]
1025it [00:17, 59.39it/s]
1089it [00:18, 59.51it/s]
1153it [00:19, 59.51it/s]
1217it [00:20, 59.64it/s]
1281it [00:21, 59.96it/s]
134

16833it [04:15, 65.92it/s]
16897it [04:16, 65.93it/s]
16961it [04:17, 65.93it/s]
17025it [04:18, 65.93it/s]
17089it [04:19, 65.92it/s]
17153it [04:20, 65.91it/s]
17217it [04:21, 65.90it/s]
17281it [04:22, 65.90it/s]
17345it [04:23, 65.90it/s]
17409it [04:24, 65.90it/s]
17473it [04:25, 65.91it/s]
17537it [04:26, 65.91it/s]
17601it [04:27, 65.90it/s]
17665it [04:28, 65.89it/s]
17729it [04:29, 65.88it/s]
17793it [04:30, 65.87it/s]
17857it [04:31, 65.88it/s]
17921it [04:32, 65.88it/s]
17985it [04:32, 65.89it/s]
18049it [04:33, 65.89it/s]
18113it [04:34, 65.90it/s]
18177it [04:35, 65.90it/s]
18241it [04:36, 65.90it/s]
18305it [04:37, 65.90it/s]
18369it [04:38, 65.90it/s]
18433it [04:39, 65.91it/s]
18497it [04:40, 65.91it/s]
18561it [04:41, 65.91it/s]
18625it [04:42, 65.91it/s]
18689it [04:43, 65.91it/s]
18753it [04:44, 65.92it/s]
18817it [04:45, 65.92it/s]
18881it [04:46, 65.92it/s]
18945it [04:47, 65.91it/s]
19009it [04:48, 65.90it/s]
19073it [04:49, 65.89it/s]
19137it [04:50, 65.89it/s]
1

...

86721it [22:53, 63.14it/s]
86785it [22:54, 63.15it/s]
86849it [22:55, 63.15it/s]
86913it [22:56, 63.15it/s]
86977it [22:57, 63.15it/s]
87041it [22:58, 63.15it/s]
87105it [22:59, 63.15it/s]
87169it [23:01, 63.12it/s]
87233it [23:02, 63.09it/s]
87297it [23:03, 63.08it/s]
87361it [23:04, 63.08it/s]
87425it [23:05, 63.09it/s]
87489it [23:06, 63.09it/s]
87553it [23:07, 63.10it/s]
87617it [23:08, 63.10it/s]
87681it [23:09, 63.11it/s]
87745it [23:10, 63.11it/s]
87809it [23:11, 63.11it/s]
87873it [23:12, 63.12it/s]
87937it [23:13, 63.12it/s]
88001it [23:14, 63.12it/s]
88065it [23:15, 63.12it/s]
88129it [23:16, 63.13it/s]
88193it [23:17, 63.13it/s]
88257it [23:17, 63.13it/s]
88321it [23:18, 63.14it/s]
88385it [23:19, 63.14it/s]
88449it [23:20, 63.15it/s]
88513it [23:21, 63.15it/s]
88577it [23:22, 63.16it/s]
88641it [23:23, 63.16it/s]
88705it [23:24, 63.16it/s]
88769it [23:25, 63.16it/s]
88833it [23:26, 63.16it/s]
88897it [23:27, 63.17it/s]
88961it [23:28, 63.17it/s]
89025it [23:29, 63.17it/s]
8

104065it [27:19, 63.48it/s]
104129it [27:20, 63.48it/s]
104193it [27:21, 63.48it/s]
104257it [27:22, 63.49it/s]
104321it [27:23, 63.49it/s]
104385it [27:24, 63.49it/s]
104449it [27:25, 63.49it/s]
104513it [27:25, 63.50it/s]
104577it [27:26, 63.50it/s]
104641it [27:27, 63.50it/s]
104705it [27:28, 63.51it/s]
104769it [27:29, 63.51it/s]
104833it [27:30, 63.51it/s]
104897it [27:31, 63.52it/s]
104961it [27:32, 63.52it/s]
105025it [27:33, 63.52it/s]
105089it [27:34, 63.52it/s]
105153it [27:35, 63.53it/s]
105217it [27:36, 63.53it/s]
105281it [27:37, 63.53it/s]
105345it [27:37, 63.54it/s]
105409it [27:38, 63.54it/s]
105473it [27:39, 63.54it/s]
105537it [27:40, 63.55it/s]
105601it [27:41, 63.55it/s]
105665it [27:42, 63.55it/s]
105729it [27:43, 63.56it/s]
105793it [27:44, 63.56it/s]
105857it [27:45, 63.56it/s]
105921it [27:46, 63.56it/s]
105985it [27:47, 63.56it/s]
106049it [27:49, 63.53it/s]
106113it [27:51, 63.50it/s]
106177it [27:52, 63.50it/s]
106241it [27:53, 63.50it/s]
106305it [27:53, 63.

120961it [31:39, 63.69it/s]
121025it [31:40, 63.69it/s]
121089it [31:41, 63.69it/s]
121153it [31:42, 63.69it/s]
121217it [31:43, 63.70it/s]
121281it [31:43, 63.70it/s]
121345it [31:44, 63.70it/s]
121409it [31:45, 63.70it/s]
121473it [31:46, 63.70it/s]
121537it [31:48, 63.69it/s]
121601it [31:49, 63.67it/s]
121665it [31:51, 63.65it/s]
121729it [31:52, 63.65it/s]
121793it [31:53, 63.65it/s]
121857it [31:54, 63.65it/s]
121921it [31:55, 63.66it/s]
121985it [31:56, 63.66it/s]
122049it [31:57, 63.66it/s]
122113it [31:58, 63.66it/s]
122177it [31:58, 63.67it/s]
122241it [31:59, 63.67it/s]
122305it [32:00, 63.67it/s]
122369it [32:01, 63.67it/s]
122433it [32:02, 63.67it/s]
122497it [32:04, 63.66it/s]
122561it [32:05, 63.64it/s]
122625it [32:07, 63.62it/s]
122689it [32:08, 63.62it/s]
122753it [32:09, 63.62it/s]
122817it [32:10, 63.63it/s]
122881it [32:11, 63.63it/s]
122945it [32:12, 63.63it/s]
123009it [32:13, 63.63it/s]
123073it [32:14, 63.64it/s]
123137it [32:14, 63.64it/s]
123201it [32:15, 63.

137857it [36:03, 63.71it/s]
137921it [36:04, 63.71it/s]
137985it [36:05, 63.72it/s]
138049it [36:06, 63.72it/s]
138113it [36:07, 63.72it/s]
138177it [36:08, 63.72it/s]
138241it [36:09, 63.72it/s]
138305it [36:10, 63.73it/s]
138369it [36:11, 63.73it/s]
138433it [36:12, 63.73it/s]
138497it [36:13, 63.73it/s]
138561it [36:14, 63.73it/s]
138625it [36:15, 63.73it/s]
138689it [36:16, 63.74it/s]
138753it [36:16, 63.74it/s]
138817it [36:17, 63.74it/s]
138881it [36:18, 63.74it/s]
138945it [36:19, 63.74it/s]
139009it [36:20, 63.75it/s]
139073it [36:21, 63.75it/s]
139137it [36:22, 63.75it/s]
139201it [36:23, 63.75it/s]
139265it [36:24, 63.76it/s]
139329it [36:25, 63.76it/s]
139393it [36:26, 63.76it/s]
139457it [36:27, 63.76it/s]
139521it [36:28, 63.77it/s]
139585it [36:29, 63.77it/s]
139649it [36:30, 63.77it/s]
139713it [36:30, 63.77it/s]
139777it [36:31, 63.77it/s]
139841it [36:32, 63.77it/s]
139905it [36:33, 63.77it/s]
139969it [36:34, 63.78it/s]
140033it [36:35, 63.78it/s]
140097it [36:36, 63.

154753it [40:24, 63.83it/s]
154817it [40:25, 63.83it/s]
154881it [40:26, 63.83it/s]
154945it [40:27, 63.83it/s]
155009it [40:28, 63.84it/s]
155073it [40:29, 63.84it/s]
155137it [40:30, 63.84it/s]
155201it [40:31, 63.84it/s]
155265it [40:32, 63.84it/s]
155329it [40:33, 63.84it/s]
155393it [40:34, 63.84it/s]
155457it [40:35, 63.84it/s]
155521it [40:35, 63.84it/s]
155585it [40:36, 63.84it/s]
155649it [40:37, 63.85it/s]
155713it [40:38, 63.85it/s]
155777it [40:39, 63.85it/s]
155841it [40:40, 63.85it/s]
155905it [40:41, 63.85it/s]
155969it [40:42, 63.85it/s]
156033it [40:43, 63.85it/s]
156097it [40:44, 63.85it/s]
156161it [40:45, 63.86it/s]
156225it [40:46, 63.86it/s]
156289it [40:47, 63.86it/s]
156353it [40:48, 63.86it/s]
156417it [40:49, 63.86it/s]
156481it [40:50, 63.86it/s]
156545it [40:51, 63.86it/s]
156609it [40:52, 63.86it/s]
156673it [40:53, 63.86it/s]
156737it [40:54, 63.86it/s]
156801it [40:55, 63.87it/s]
156865it [40:56, 63.87it/s]
156929it [40:57, 63.87it/s]
156993it [40:58, 63.